In [41]:
import os
import pandas as pd
import numpy as np
import pathlib
import xarray as xr
import rioxarray

In [42]:
dir=pathlib.Path('./../data_global_surge/merage/')

In [43]:
file_names = [f.name for f in dir.iterdir() if f.is_file()]

In [44]:
file_names[0]

'field_2024-11-05_00_00_00.f0.nc'

In [45]:
# 'field_2024-11-05_00_00_00.f0.nc'
file_name='field_2024-11-05_00_00_00.f0.nc'
file_name.split('.')[0].split('_')

['field', '2024-11-05', '00', '00', '00']

In [46]:
# 提取时间信息的函数
def extract_time_from_filename(filename):
    """
    从文件名中提取时间信息，返回一个 numpy.datetime64 对象。
    假设文件名格式为 field_YYYY-MM-DD_HH_MM.nc
    """
    name_splits=file_name.split('.')[0].split('_')
    yyymmdd=name_splits[1]
    hh=name_splits[2]
    dt_str=f'{yyymmdd}T{hh}:00:00'
    return np.datetime64(dt_str)

In [26]:
extract_time_from_filename('field_2024-11-05_00_00_00.f0.nc')

numpy.datetime64('2024-11-05T00:00:00')

In [ ]:
pathlib.Path(dir) / file_names[0]

WindowsPath('../data_global_surge/merage/field_2024-11-05_00_00_00.f0.nc')

In [29]:
ds = xr.open_dataset(str(pathlib.Path(dir) / file_names[0]))

In [30]:
ds

<xarray.Dataset>
Dimensions:    (Time: 1, latitude: 1001, longitude: 901)
Coordinates:
  * longitude  (longitude) float64 100.0 100.0 100.1 100.2 ... 144.9 144.9 145.0
  * latitude   (latitude) float64 5.0 5.05 5.1 5.15 ... 54.85 54.9 54.95 55.0
  * Time       (Time) datetime64[ns] 2024-11-05
Data variables:
    h          (Time, latitude, longitude) float32 ...
    maskLand   (latitude, longitude) float64 ...

In [ ]:
datasets = []

for file in file_names:
    file_info = pathlib.Path(dir) / file
    file_fullpath: str = str(file_info)
    # 打开 NetCDF 文件
    ds = xr.open_dataset(file_fullpath)

    # 提取时间信息
    time = extract_time_from_filename(file_fullpath)

    # 扩展一个时间维度，并赋值
    ds = ds.expand_dims(dim={"time": [time]})

    # 添加到列表中
    datasets.append(ds)

# 合并所有数据集
combined_ds = xr.concat(datasets, dim="time")

# 保存到新的 NetCDF 文件
# combined_ds.to_netcdf("combined_file.nc")

In [18]:
combined_ds.to_netcdf("combined_file.nc")

KeyboardInterrupt: 

#### 目标:  
从多个 NetCDF 文件中读取 h 变量，并将 Time 维度合并为一个完整的时间数组，最终拼接为一个多维数组。

##### step-1 文件读取：

使用 `xr.open_dataset()` 打开每个 NetCDF 文件。
检查是否包含 h 变量和 `Time` 维度，确保数据格式正确。
##### step-2 拼接数据：

使用 `xr.concat()` 按 `Time` 维度拼接多个文件的数据。
`xr.concat()` 会自动将每个文件的 `Time` 维度合并为一个完整的时间数组。
##### step-3 保存结果：

拼接后的数据保存为新的 NetCDF 文件 `combined_h.nc`。
##### step-4 时间数组：

拼接完成后，`combined_ds["Time"]` 是一个包含所有时间点的数组，可以通过 `print()` 查看。

#### 注意事项
#####  时间维度对齐：

如果文件中的 `Time` 维度值不连续或存在重复，`xr.concat()` 会自动对齐时间点，但可能需要手动检查时间顺序。
#####  维度一致性：

确保所有文件的 `lat` 和 `lon` 维度一致（如分辨率、范围相同），否则会引发拼接错误。
#####  内存优化：

如果文件较大，可以使用 `dask` 进行分块处理，例如：
`ds = xr.open_dataset(file, chunks={"Time": 1})`

#####  变量选择：

如果只需要` h` 变量，可以在打开文件时直接选择：
`ds = xr.open_dataset(file)[["h"]]`


In [47]:
# 创建一个空列表，用于存储每个文件的 Dataset
datasets = []

# 遍历文件列表，读取每个文件的数据
for file in file_names:
    file_info = pathlib.Path(dir) / file
    file_fullpath: str = str(file_info)
    # 打开 NetCDF 文件
    ds = xr.open_dataset(file_fullpath)
    
    # 检查文件中是否包含 h 和 Time
    if "h" in ds.variables and "Time" in ds.dims:
        datasets.append(ds)
    else:
        print(f"文件 {file_fullpath} 中缺少 'h' 变量或 'Time' 维度，跳过处理。")

# 拼接所有数据集
if datasets:
    # 使用 xarray.concat 按 Time 维度拼接
    combined_ds = xr.concat(datasets, dim="Time")
    
    # 打印拼接后的时间数组
    print("Time 维度的数组：")
    print(combined_ds["Time"].values)
    
    # 保存拼接后的数据到新的 NetCDF 文件
    combined_ds.to_netcdf("combined_h.nc")
    print("拼接完成，结果保存为 combined_h.nc")
else:
    print("未找到可用的数据，未进行拼接。")

Time 维度的数组：
['2024-11-05T00:00:00.000000000' '2024-11-05T01:00:00.000000000'
 '2024-11-05T02:00:00.000000000' '2024-11-05T03:00:00.000000000'
 '2024-11-05T04:00:00.000000000' '2024-11-05T05:00:00.000000000'
 '2024-11-05T06:00:00.000000000' '2024-11-05T07:00:00.000000000'
 '2024-11-05T08:00:00.000000000' '2024-11-05T09:00:00.000000000'
 '2024-11-05T10:00:00.000000000' '2024-11-05T11:00:00.000000000'
 '2024-11-05T12:00:00.000000000' '2024-11-05T13:00:00.000000000'
 '2024-11-05T14:00:00.000000000' '2024-11-05T15:00:00.000000000'
 '2024-11-05T16:00:00.000000000' '2024-11-05T17:00:00.000000000'
 '2024-11-05T18:00:00.000000000' '2024-11-05T19:00:00.000000000'
 '2024-11-05T20:00:00.000000000' '2024-11-05T21:00:00.000000000'
 '2024-11-05T22:00:00.000000000' '2024-11-05T23:00:00.000000000'
 '2024-11-06T00:00:00.000000000' '2024-11-06T01:00:00.000000000'
 '2024-11-06T02:00:00.000000000' '2024-11-06T03:00:00.000000000'
 '2024-11-06T04:00:00.000000000' '2024-11-06T05:00:00.000000000'
 '2024-11-06T